# Description

**TODO**

# Modules

In [1]:
library(reticulate)
pd <- import("pandas")

# Settings

In [2]:
# reference panel
REFERENCE_PANEL <- "GTEX_V8"
# REFERENCE_PANEL = "1000G"

# prediction models
## mashr
EQTL_MODEL <- "MASHR"

# ## elastic net
# EQTL_MODEL = "ELASTIC_NET"

chromosome <- NULL

In [3]:
# Parameters
chromosome = 4
REFERENCE_PANEL = "GTEX_V8"
EQTL_MODEL = "MASHR"


In [4]:
paste0("Using reference panel: ", REFERENCE_PANEL)

[1] "Using reference panel: GTEX_V8"

In [5]:
paste0("Using eQTL model: ", EQTL_MODEL)

[1] "Using eQTL model: MASHR"

In [6]:
# chromosome must be provided as parameter
stopifnot(!is.null(chromosome))

# Paths

In [7]:
GENE_CORRS_DIR <- Sys.getenv("PHENOPLIER_PHENOMEXCAN_LD_BLOCKS_GENE_CORRS_DIR")
IRdisplay::display(GENE_CORRS_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs"

In [8]:
INPUT_DIR <- file.path(GENE_CORRS_DIR, tolower(REFERENCE_PANEL), tolower(EQTL_MODEL), "by_chr")
IRdisplay::display(INPUT_DIR)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr"

In [9]:
INPUT_FILE <- file.path(INPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(INPUT_FILE)
stopifnot(file.exists(INPUT_FILE))

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/gene_corrs-chr4.pkl"

In [10]:
OUTPUT_DIR <- file.path(INPUT_DIR, "corrected_positive_definite")
IRdisplay::display(OUTPUT_DIR)
dir.create(OUTPUT_DIR, recursive = TRUE, showWarnings = FALSE)

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite"

In [11]:
OUTPUT_FILE <- file.path(OUTPUT_DIR, paste0("gene_corrs-chr", chromosome, ".pkl"))
IRdisplay::display(OUTPUT_FILE)
if (file.exists(OUTPUT_FILE)) {
  IRdisplay::display("Output file exists, it will be overwritten")
}

[1] "/opt/data/data/phenomexcan/ld_blocks/gene_corrs/gtex_v8/mashr/by_chr/corrected_positive_definite/gene_corrs-chr4.pkl"

[1] "Output file exists, it will be overwritten"

# Functions

In [12]:
# taken from https://www.r-bloggers.com/2013/08/correcting-a-pseudo-correlation-matrix-to-be-positive-semidefinite/
# TODO: add documentation
CorrectCM <- function(CM, p = 0) {
  n <- dim(var(CM))[1L]
  E <- eigen(CM)
  CM1 <- E$vectors %*% tcrossprod(diag(pmax(E$values, p), n), E$vectors)
  Balance <- diag(1 / sqrt(diag(CM1)))
  CM2 <- Balance %*% CM1 %*% Balance
  return(CM2)
}

In [13]:
is_positive_definite <- function(data) {
  eigenvalues <- eigen(data)$values
  nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]

  if (length(nonpositive_eigenvalues) > 0) {
    IRdisplay::display("We need to correct the data and make the matrix positive definite")
    return(FALSE)
  } else {
    IRdisplay::display("Matrix is already positive definite!")
    return(TRUE)
  }
}

# Load data

In [14]:
gene_corrs <- pd$read_pickle(INPUT_FILE)

In [15]:
dim(gene_corrs)

[1] 244 244

In [16]:
head(gene_corrs[1:10, 1:10])

,ENSG00000272602,ENSG00000131127,ENSG00000174227,ENSG00000133256,ENSG00000169020,ENSG00000215375,ENSG00000169026,ENSG00000178950,ENSG00000145214,ENSG00000145217
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000272602,1.000000000,-0.009553751,0.150293675,0.03335459,-0.14932365,-0.01873805,0.22156662,-0.165014736,0.08899452,-0.02932051
ENSG00000131127,-0.009553751,1.000000000,0.007011144,-0.01196897,-0.11262825,-0.33456193,0.47397527,0.006672161,-0.17425058,-0.08271097
ENSG00000174227,0.150293675,0.007011144,1.000000000,0.15153585,-0.25450142,-0.07142492,0.39495518,0.059066888,-0.04577018,0.19981893
ENSG00000133256,0.033354592,-0.011968967,0.151535848,1.00000000,0.09553455,-0.16724901,-0.02378855,0.070181002,0.10198709,0.02238429
ENSG00000169020,-0.149323648,-0.112628250,-0.254501417,0.09553455,1.00000000,0.01104808,-0.14681850,-0.160465205,0.11454032,0.01720044
ENSG00000215375,-0.018738046,-0.334561933,-0.071424922,-0.16724901,0.01104808,1.00000000,-0.30027667,0.025483868,-0.10166441,0.19250162


# Check positive definiteness

In [17]:
is_positive_definite(gene_corrs)

[1] "We need to correct the data and make the matrix positive definite"

[1] FALSE

In [18]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs)$values

In [19]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 11

[1] -0.02285956 -0.02384652 -0.06264010 -0.06652011 -0.07954689 -0.17066973
 [7] -0.17856772 -0.19521199 -0.36536244 -0.61137324 -0.89465733

In [20]:
if (length(eigenvalues[eigenvalues <= 0]) == 0) { quit() }

# Make matrix positive definite if needed

In [21]:
gene_corrs_corrected <- CorrectCM(gene_corrs, 1)

In [22]:
dimnames(gene_corrs_corrected)[[1]] <- rownames(gene_corrs)

In [23]:
dimnames(gene_corrs_corrected)[[2]] <- colnames(gene_corrs)

In [24]:
gene_corrs_corrected <- as.data.frame(gene_corrs_corrected)

In [25]:
dim(gene_corrs_corrected)

[1] 244 244

# Check positive definiteness of corrected matrix

In [26]:
is_positive_definite(gene_corrs_corrected)

[1] "Matrix is already positive definite!"

[1] TRUE

In [27]:
# see eigenvalues
eigenvalues <- eigen(gene_corrs_corrected)$values

In [28]:
nonpositive_eigenvalues <- eigenvalues[eigenvalues <= 0]
IRdisplay::display(length(nonpositive_eigenvalues))
IRdisplay::display(nonpositive_eigenvalues)

[1] 0

numeric(0)

In [29]:
stopifnot(length(eigenvalues[eigenvalues <= 0]) == 0)

In [30]:
# quick and visual comparison of the two matrices
IRdisplay::display(head(gene_corrs[1:10, 1:10]))
IRdisplay::display(head(gene_corrs_corrected[1:10, 1:10]))

,ENSG00000272602,ENSG00000131127,ENSG00000174227,ENSG00000133256,ENSG00000169020,ENSG00000215375,ENSG00000169026,ENSG00000178950,ENSG00000145214,ENSG00000145217
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000272602,1.000000000,-0.009553751,0.150293675,0.03335459,-0.14932365,-0.01873805,0.22156662,-0.165014736,0.08899452,-0.02932051
ENSG00000131127,-0.009553751,1.000000000,0.007011144,-0.01196897,-0.11262825,-0.33456193,0.47397527,0.006672161,-0.17425058,-0.08271097
ENSG00000174227,0.150293675,0.007011144,1.000000000,0.15153585,-0.25450142,-0.07142492,0.39495518,0.059066888,-0.04577018,0.19981893
ENSG00000133256,0.033354592,-0.011968967,0.151535848,1.00000000,0.09553455,-0.16724901,-0.02378855,0.070181002,0.10198709,0.02238429
ENSG00000169020,-0.149323648,-0.112628250,-0.254501417,0.09553455,1.00000000,0.01104808,-0.14681850,-0.160465205,0.11454032,0.01720044
ENSG00000215375,-0.018738046,-0.334561933,-0.071424922,-0.16724901,0.01104808,1.00000000,-0.30027667,0.025483868,-0.10166441,0.19250162


,ENSG00000272602,ENSG00000131127,ENSG00000174227,ENSG00000133256,ENSG00000169020,ENSG00000215375,ENSG00000169026,ENSG00000178950,ENSG00000145214,ENSG00000145217
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000272602,1.000000000,0.028396694,0.08695441,0.002295969,-0.08346763,-0.04525615,0.07294869,-0.05899910,0.038594126,-0.029559569
ENSG00000131127,0.028396694,1.000000000,0.05617119,-0.002971049,-0.04778612,-0.16047656,0.18112620,0.01679856,-0.076828257,-0.006607089
ENSG00000174227,0.086954408,0.056171185,1.00000000,0.046791821,-0.12264813,-0.04778183,0.17567495,0.06052697,-0.005914992,0.109970559
ENSG00000133256,0.002295969,-0.002971049,0.04679182,1.000000000,0.03491539,-0.08277364,0.02635418,0.03970548,0.063155939,0.021925123
ENSG00000169020,-0.083467632,-0.047786118,-0.12264813,0.034915394,1.00000000,0.02232428,-0.12997424,-0.04662427,0.036021650,-0.039444413
ENSG00000215375,-0.045256148,-0.160476561,-0.04778183,-0.082773639,0.02232428,1.00000000,-0.16083800,0.02922385,-0.061470865,0.096611995


Both matrices should "look" similar. We are not interested in perfectly accurate correlation values (they are already inaccurate).

# Save

In [31]:
py_save_object(gene_corrs_corrected, OUTPUT_FILE)